In [ ]:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import os
import pandas as pd

# Configurar opciones de Chrome
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Ejecutar en modo headless
chrome_options.add_argument("--start-maximized")  # Abrir el navegador en pantalla completa
# Establecer el perfil de usuario
user_data_dir = "chrome_user_data"
chrome_options.add_argument(f"user-data-dir={user_data_dir}")

# Descargar y configurar el controlador de Chrome automáticamente
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Abrir el navegador y navegar a Apollo.io
driver.get("https://app.apollo.io/#/login")

# Esperar unos segundos para ver el navegador abierto antes de empezar a interactuar
time.sleep(1)

# Intentar encontrar el campo de correo electrónico por su XPath y rellenarlo
email_xpath = '/html/body/div[2]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div/form/div[2]/div/div/input'
try:
    email_input = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, email_xpath))
    )
    email_input.send_keys("Gbustosgarcia01@gmail.com")
    print("El campo de correo electrónico fue rellenado.")
except Exception as e:
    print(f"El campo de correo electrónico no fue encontrado: {e}")

# Intentar encontrar el campo de contraseña por su XPath y rellenarlo
password_xpath = '/html/body/div[2]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div/form/div[3]/div/div[1]/div/input'
try:
    password_input = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, password_xpath))
    )
    password_input.send_keys("LG030920..lg.")
    print("El campo de contraseña fue rellenado.")
except Exception as e:
    print(f"El campo de contraseña no fue encontrado: {e}")

# Intentar encontrar el botón de Log In por su XPath y hacer clic en él
login_button_xpath = '/html/body/div[2]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div/form/div[4]/button'
try:
    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, login_button_xpath))
    )
    login_button.click()
    print("El botón de Log In fue clickeado.")
except Exception as e:
    print(f"El botón de Log In no fue encontrado o no fue clickeable: {e}")

# Esperar unos segundos para asegurar que el inicio de sesión se complete
time.sleep(5)

# Cerrar el navegador
driver.quit()


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import csv
import time

# Configurar opciones de Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecutar en modo headless
chrome_options.add_argument("--start-maximized")  # Abrir el navegador en pantalla completa
user_data_dir = "chrome_user_data"
chrome_options.add_argument(f"user-data-dir={user_data_dir}")

# Descargar y configurar el controlador de Chrome automáticamente
def start_driver():
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=chrome_options)

driver = start_driver()

csv_file_path = 'Resultados/Empresas/Apollo_Acounts_1-10.csv'
rows = []
company_urls = {}  # Diccionario para mapear URLs a sus índices de fila

# Verificar y añadir columnas si no existen
with open(csv_file_path, mode='r', newline='', encoding='utf-8') as file:
    csv_reader = csv.DictReader(file)
    existing_fieldnames = csv_reader.fieldnames
    required_columns = ['Website', 'LinkeIn', 'Faceboock', 'Twitter', 'Descripción', 'Processed']
    
    if not all(column in existing_fieldnames for column in required_columns):
        fieldnames = existing_fieldnames + [column for column in required_columns if column not in existing_fieldnames]
    else:
        fieldnames = existing_fieldnames

    for index, row in enumerate(csv_reader):
        rows.append(row)
        company_urls[row.get('Company URL')] = index  # Guardar el índice de fila por URL

num_rows_to_process = 1000  # Cambia este valor según sea necesario
processed_rows = 0
for row_index, row in enumerate(rows):
    if processed_rows >= num_rows_to_process:
        break

    if row.get('Processed') == 'yes':
        print(f"Fila {row_index + 1} ya está completa. Pasando a la siguiente fila.")
        continue

    if any([not row.get(col, '').strip() for col in required_columns if col != 'Processed']):
        company_name = row.get('Company', '')
        company_url = row.get('Company URL', '')

        print(f"Procesando fila {row_index + 1} - Empresa: {company_name}")

        try:
            driver.get(company_url)
            time.sleep(2)

            div_xpath = '/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div[1]/div[1]/div[1]/div/div/div/div/div[1]/div[2]'
            links = {
                'Website': row.get('Website', 'No tiene Website'),
                'LinkeIn': row.get('LinkeIn', 'No tiene LinkeIn'),
                'Faceboock': row.get('Faceboock', 'No tiene Faceboock'),
                'Twitter': row.get('Twitter', 'No tiene Twitter'),
                'Descripción': row.get('Descripción', 'No tiene descripción'),
            }

            if any([not row.get(col, '').strip() for col in required_columns if col != 'Processed']):
                try:
                    div_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, div_xpath))
                    )
                    link_elements = div_element.find_elements(By.TAG_NAME, 'a')
                    
                    for link in link_elements:
                        href = link.get_attribute('href')
                        if href:
                            if 'linkedin.com' in href:
                                links['LinkeIn'] = href
                            elif 'facebook.com' in href:
                                links['Faceboock'] = href
                            elif 'twitter.com' in href or 'x.com' in href:
                                links['Twitter'] = href
                            else:
                                if links['Website'] == 'No tiene Website':
                                    links['Website'] = href
                
                except Exception as e:
                    print(f"No se pudo encontrar el div con enlaces: {e}")

                show_more_xpath = '/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div[1]/div[1]/div[2]/div/div/div/div/div/div[1]/div/div/div/span/a'
                try:
                    show_more_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, show_more_xpath))
                    )
                    show_more_button.click()
                    
                    time.sleep(2)
                    
                    description_xpath = '/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div[1]/div[1]/div[2]/div/div/div/div/div/div[1]/div/div/div/span/span'
                    try:
                        description_element = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, description_xpath))
                        )
                        description = description_element.text
                        if description:
                            links['Descripción'] = description
                        else:
                            links['Descripción'] = 'No tiene descripción'
                        
                    except Exception as e:
                        links['Descripción'] = 'No tiene descripción'
                        
                except Exception as e:
                    links['Descripción'] = 'No tiene descripción'

            rows[row_index].update(links)
            rows[row_index]['Processed'] = 'yes'
            processed_rows += 1

        except Exception as e:
            print(f"Error al procesar la URL {company_url}: {e}")
            rows[row_index].update({
                'Website': 'Error de página',
                'LinkeIn': 'Error de página',
                'Faceboock': 'Error de página',
                'Twitter': 'Error de página',
                'Descripción': 'Error de página',
                'Processed': 'yes'
            })
            processed_rows += 1
            # Cerrar el navegador y reiniciar
            driver.quit()
            driver = start_driver()

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as updated_file:
    csv_writer = csv.DictWriter(updated_file, fieldnames=fieldnames)
    csv_writer.writeheader()
    csv_writer.writerows(rows)

print(f"{csv_file_path} actualizado correctamente")
driver.quit()